In [13]:
import pandas as pd
import numpy as np
import datetime
import logger

In [14]:
dates = []
date  = datetime.datetime(2019, 5, 15)

for i in range(17):

    if(i%2 == 0):
        ndate = date + datetime.timedelta(hours=0, minutes = 0, seconds = 20)
    elif(i%3 == 0):
        ndate = date - datetime.timedelta(hours=0, minutes = 0, seconds = 2)
    elif(i%4 == 0):
        ndate = date - datetime.timedelta(hours=0, minutes = 0, seconds = 10)
    elif(i%5 == 0):
        ndate = date - datetime.timedelta(hours=0, minutes = 0, seconds = 3)
    else:
        ndate = date + datetime.timedelta(hours=0 ,minutes=0, seconds = 0)
        
    dates.append(ndate)
    date = ndate

dates

df = pd.DataFrame(np.random.randn(17, 4), index=dates, columns=['A', 'B', 'C', 'D'])
print(df)
print(df.columns)

                            A         B         C         D
2019-05-15 00:00:20 -0.632673 -0.076356  0.740482 -1.293758
2019-05-15 00:00:20 -0.629153  0.142680  1.197175  1.426024
2019-05-15 00:00:40  0.014624 -0.939138  0.379475  1.238219
2019-05-15 00:00:38 -0.057139  1.169432  0.991922 -1.161098
2019-05-15 00:00:58  1.140074 -1.870670  0.148647  0.160543
2019-05-15 00:00:55 -0.623893 -0.821005 -0.156214  0.364467
2019-05-15 00:01:15  0.407902 -0.082176 -0.336603 -0.495715
2019-05-15 00:01:15  1.089271  0.682740 -0.569875  1.473240
2019-05-15 00:01:35 -0.755965 -0.734179 -0.645755  0.761400
2019-05-15 00:01:33  0.474224 -0.365034  0.564351 -1.815273
2019-05-15 00:01:53 -1.159814  0.079918  0.907609  1.376399
2019-05-15 00:01:53 -0.646965  1.257073  0.974703  0.198590
2019-05-15 00:02:13  0.482354  1.012595  2.127562  0.543398
2019-05-15 00:02:13  0.919450  0.178667 -0.681692  0.436830
2019-05-15 00:02:33 -0.786295 -0.028471 -0.205520  0.758686
2019-05-15 00:02:31 -0.169933  0.494694 

In [15]:
# converting timestamp column values into datetime type which may come in string format and then sorting the dates and setting dates column as index
def clean_ts_col(df, ts_col):
    # Sort timestamp and set timestamp as index
    df[ts_col] = pd.to_datetime(df[ts_col])
    df = df.sort_values(ts_col).set_index(ts_col)
    return df

In [22]:
def get_metrics_ts(df, host_name):
    # Remove empty columns.
    df = df.dropna(axis=1, thresh=1)
    logger.logger.info('Info: Empty columns dropped.')
    
    # Convert numeric columns to float where possible, since data came in as strings (JSON).
    cols_to_convert = [c for c in df.columns if c!= host_name]
    for c in cols_to_convert:
        try:
            df[c] = df[c].astype(float)
            # logger.info('Info: Data type converted to numeric.')
        except:
            logger.logger.warning('Warning: Data type cannot be converted to numeric in column {}.'.format(c))
            pass
        
    # Break each column, remove missing values, return as a list
    try:
        # Drop the missing value for each column
        vm_metrics_list = [df[c].dropna() for c in df.columns]
        # logger.info('Info: Sparse columns are converted to a list of time series for this node.')
    except:
        logger.exception('Error: Cannot convert sparse columns into time series.')
    return vm_metrics_list

In [23]:
def find_sampling_freq(ts):
    # assert ts.index is datetime object
    try:
        # For non-NA values, check time difference in seconds in each timestamp record for each metric
        sf = pd.Series(ts.index).diff().dropna().dt.seconds
        # logger.info('Info: Found natural sampling frequency')
    except:
        logger.exception('Error: Cannot find natural sampling frequency.')
        
    # Only keep record with timestamp difference greater than 0 seconds
    sf = sf[sf > 0]
    # if the series has different timestamp, take the mininum value from the mode as the sampling frequency
    if len(sf) > 0:
        sampling_frequency = int(sf.mode().values.min())
    # Otherwise, sampling frequency is 0
    else:
        sampling_frequency = 0
    return ts, sampling_frequency

In [27]:
def aggregate_ts(ts, orig_frequency, new_frequency):
    # Compare the calculated sampling frequency with the defined re-sample frequency
    # resample
    print('Original frequency:', orig_frequency)
    print('New frequency:', new_frequency)
    if orig_frequency <= new_frequency:
        # print(ts.name, new_frequency)
        if ts.dtypes != 'object':
            ts_new = ts.resample(str(new_frequency)+'s').first().interpolate()
        else:
            ts_new = ts.resample(str(new_frequency)+'s').first()
    else:
        # print(ts.name, orig_frequency)
        if ts.dtypes != 'object':
            ts_new = ts.resample(str(orig_frequency)+'s').first().interpolate()
        else:
            ts_new = ts.resample(str(orig_frequency)+'s').first()
    # print(ts_new.name, ts_new.dtypes)
    return ts_new

In [28]:
def preprocess_by_endpoint(vm, ts_col, host_name, new_frequency):
    # each 'vm' is a df of relevant metrics for an individual endpoint
    # Sort timestamp and set timestamp as index
    # vm = clean_ts_col(vm, ts_col)
    # Convert numeric columns to float if necessary, drop null value for each column and return a list of list for each metric
    vm_metrics_list = get_metrics_ts(vm, host_name)
    # For each metric, compute the sampling frequency. It returns vm_metrics_list and its corresponding sampling frequency
    samp_freqs = [find_sampling_freq(met) for met in vm_metrics_list]
    # Compare the calculated sampling frequency with the pre-defined re-sample frequency, resample and get new vm_metrics_list
    vm_metrics_list = [aggregate_ts(ts, of, new_frequency) for ts, of in samp_freqs]
    return vm_metrics_list

In [29]:
preprocess_by_endpoint(df,'no ts column','no host name', 30)

[15/May/2019 18:21:43] INFO - Info: Empty columns dropped.
Original frequency: 20
New frequency: 30
Original frequency: 20
New frequency: 30
Original frequency: 20
New frequency: 30
Original frequency: 20
New frequency: 30


[2019-05-15 00:00:00   -0.632673
 2019-05-15 00:00:30   -0.057139
 2019-05-15 00:01:00    0.407902
 2019-05-15 00:01:30    0.474224
 2019-05-15 00:02:00    0.482354
 2019-05-15 00:02:30   -0.169933
 Freq: 30S, Name: A, dtype: float64, 2019-05-15 00:00:00   -0.076356
 2019-05-15 00:00:30    1.169432
 2019-05-15 00:01:00   -0.082176
 2019-05-15 00:01:30   -0.365034
 2019-05-15 00:02:00    1.012595
 2019-05-15 00:02:30    0.494694
 Freq: 30S, Name: B, dtype: float64, 2019-05-15 00:00:00    0.740482
 2019-05-15 00:00:30    0.991922
 2019-05-15 00:01:00   -0.336603
 2019-05-15 00:01:30    0.564351
 2019-05-15 00:02:00    2.127562
 2019-05-15 00:02:30    0.287569
 Freq: 30S, Name: C, dtype: float64, 2019-05-15 00:00:00   -1.293758
 2019-05-15 00:00:30   -1.161098
 2019-05-15 00:01:00   -0.495715
 2019-05-15 00:01:30   -1.815273
 2019-05-15 00:02:00    0.543398
 2019-05-15 00:02:30    1.518251
 Freq: 30S, Name: D, dtype: float64]